# Federated K-Means Clustering with Scikit-learn on Iris Dataset

## Introduction to Scikit-learn, tabular data, and federated k-Means
### Scikit-learn
This example shows how to use [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) on tabular data.
It uses [Scikit-learn](https://scikit-learn.org/),
a widely used open-source machine learning library that supports supervised 
and unsupervised learning.
### Tabular data
The data used in this example is tabular in a format that can be handled by [pandas](https://pandas.pydata.org/), such that:
- rows correspond to data samples
- the first column represents the label 
- the other columns cover the features.    

Each client is expected to have one local data file containing both training 
and validation samples. To load the data for each client, the following 
parameters are expected by the local learner:
- data_file_path: string, the full path to the client's data file 
- train_start: int, start row index for the training set
- train_end: int, end row index for the training set
- valid_start: int, start row index for the validation set
- valid_end: int, end row index for the validation set

### Federated k-Means clustering
The machine learning algorithm in this example is [k-Means clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).
The aggregation follows the scheme defined in [Mini-batch k-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html). 
Under this setting, each round of federated learning can be formulated as follows:
- local training: starting from global centers, each client trains a local MiniBatchKMeans model with their own data
- global aggregation: server collects the cluster center, 
  counts information from all clients, aggregates them by considering 
  each client's results as a mini-batch, and updates the global center and per-center counts.

For center initialization, at the first round, each client generates its initial centers with the k-means++ method. Then, the server collects all initial centers and performs one round of k-means to generate the initial global center.

Below we listed steps to run this example.

## 1. Setup NVFLARE

Follow the [Getting_Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to setup virtual environment and install NVFLARE

We also provide a [Notebook](../../nvflare_setup.ipynb) for this setup process. 

Assume you have already setup the venv, lets first install required packages.

In [6]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 2. Data preparation 
This example uses the Iris dataset available from Scikit-learn's dataset API.  

In [7]:
%env DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv
!python3 ./utils/prepare_data.py --dataset_name iris --randomize 1 --out_path ${DATASET_PATH}

env: DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv


## 3. Prepare clients' configs with proper data split information 
We are using NVFlare's FL simulator to run the following experiments. Here we simulate 3 clients with uniform data split. Since the dataset is already randomized in the last step, the split is done sequentially. For unsupervised clustering, we use the whole dataset with ground truth label to validate the performance.

In [8]:
%env DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv
!python3 ./utils/prepare_job_config.py --task_name "sklearn_kmeans" --data_path "${DATASET_PATH}" --site_num 3 --valid_frac 1 --split_method "uniform"

env: DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv


## 4. Run simulated kmeans experiment
Now that we have the job configs ready, we run the experiment using Simulator.

Simulator can either be used with CLI command: 

In [9]:
! nvflare simulator ./jobs/sklearn_kmeans_3_uniform -w /tmp/nvflare/sklearn_kmeans_iris -n 3 -t 3

2023-03-30 19:50:33,732 - SimulatorRunner - INFO - Create the Simulator Server.
2023-03-30 19:50:33,734 - Cell - INFO - server: creating listener on tcp://0:39023
2023-03-30 19:50:33,734 - Cell - INFO - server: created backbone external listener for tcp://0:39023
2023-03-30 19:50:33,735 - ConnectorManager - INFO - 11565: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-03-30 19:50:33,735 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE uds:///tmp/nvflare_334564] is starting
2023-03-30 19:50:34,236 - Cell - INFO - server: created backbone internal listener for uds:///tmp/nvflare_334564
2023-03-30 19:50:34,236 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:39023] is starting
2023-03-30 19:50:34,316 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 42357
2023-03-30 19:50:34,316 - SimulatorRunner - INFO - Deploy the Apps.
2023-03-30 19:50:34,320 - SimulatorRunner - INFO - Cr

or via Simulator API:

In [ ]:
import os
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner  

simulator = SimulatorRunner(
    job_folder=f"./jobs/sklearn_kmeans_3_uniform",
    workspace="/tmp/nvflare/sklearn_kmeans_iris",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

## 5. Result visualization
Model accuracy is computed as the homogeneity score between the cluster formed and the ground truth label, which can be visualized in tensorboard.

In [10]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/sklearn_kmeans_iris

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7654), started 2:14:05 ago. (Use '!kill 7654' to kill it.)